In [2]:
import pandas as pd
import numpy as np
import joblib

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler


In [4]:
# raw_data = pd.read_csv(r"../data/processed_data/undersampled_train.csv", index_col=0)
raw_data = pd.read_csv(r"../data/processed_data/oversampled_train.csv", index_col=0)


In [5]:
x = raw_data.drop(["TARGET"], axis=1)
y = raw_data.filter(["TARGET"], axis=1)

scaler = (
    StandardScaler()
)  # MinMaxScaler often made values = 0.0, which = lost features in training
x_scaled = scaler.fit_transform(x)
y = y.values.reshape(y.shape[0])

x_scaled.shape, y.shape


((480566, 120), (480566,))

In [7]:
clf = LinearDiscriminantAnalysis().fit(x_scaled, y)


In [8]:
joblib_file = "LinearDiscriminantAnalysis_v1.01.pkl"
joblib.dump(clf, joblib_file)


['LinearDiscriminantAnalysis_v1.01.pkl']